## Comparison of Concept Frames between Base Model and Instruction-finetuned Model

We analyze how the concepts change from base to instruction model. 

Although the unemebedding vectors can be very different between models, the concepts
stay almost the same.

In [1]:
import pandas as pd
import torch

from frames.representations import FrameUnembeddingRepresentation
from frames.utils.memory import gc_cuda
from frames.utils.plotting import histplot_and_save

In [ ]:
MODEL_IDS = {
    # model family | model ids
    # Llama 3.1
    "Llama 3.1 70B (Base x Instruct)": (
        "hugging-quants/Meta-Llama-3.1-70B-BNB-NF4-BF16",
        "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
    ),
    "Llama 3.1 8B (Base x Instruct)": (
        "meta-llama/Meta-Llama-3.1-8B",
        "meta-llama/Meta-Llama-3.1-8B-Instruct",
    ),
    # Gemma 2
    "Gemma 2 27B (Base x Instruct)": ("google/gemma-2-27b", "google/gemma-2-27b-it"),
    "Gemma 2 9B (Base x Instruct)": ("google/gemma-2-9b", "google/gemma-2-9b-it"),
}

LEMMA_COUNT = 3  # {"gemma": 25, "llama": 55}
FRAME_RANK = 3

VALUE = "Similarity"
HUE = "Model Family"

In [ ]:
def get_concepts(model_id):
    with gc_cuda():
        return FrameUnembeddingRepresentation.from_model_id(
            model_id, torch_dtype=torch.bfloat16
        ).get_all_concepts(
            min_lemmas_per_synset=LEMMA_COUNT, max_token_count=FRAME_RANK
        )


def get_similarities(id1, id2):
    return get_concepts(id1).similarity(get_concepts(id2)).float().cpu()


df = pd.concat(
    [
        pd.DataFrame({model_family: get_similarities(*model_ids)}).melt(
            var_name=HUE, value_name=VALUE
        )
        for model_family, model_ids in MODEL_IDS.items()
    ]
)

histplot_and_save("04_model_family_concept_space_comparison", df, VALUE, HUE)